In [2]:
#  import packages
import pandas as pd

In [3]:
#  import data
data_raw = pd.read_csv('../Project_datasets/campaign_orders.csv')

In [4]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436577 entries, 0 to 436576
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   user_id                          436577 non-null  int64  
 1   product_id                       436577 non-null  int64  
 2   total_cost_pennies               436577 non-null  int64  
 3   source                           436577 non-null  object 
 4   total_cost_pre_discount_pennies  436577 non-null  int64  
 5   discount%                        436577 non-null  float64
 6   campaign_id                      436577 non-null  object 
 7   sent_at                          436577 non-null  object 
 8   name_campaigns                   391693 non-null  object 
 9   opened_email                     126148 non-null  object 
 10  purchase                         436577 non-null  int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 36.6+ MB


In [5]:
data_raw.head()

,user_id,product_id,total_cost_pennies,source,total_cost_pre_discount_pennies,discount%,campaign_id,sent_at,name_campaigns,opened_email,purchase
0,386193,1782,1870,web,2200,15.0,28c8c1c87f0336332ee2a1074894eb1c,2017-02-23,New template,True,0
1,386193,1782,1870,web,2200,15.0,2c7f0e90318c992d9e5c869a396e5825,2017-03-05,NaN,NaN,0
2,386193,1782,1870,web,2200,15.0,eaeefcf45446021f396ff0d63c57983a,2017-03-09,NaN,True,0
3,386193,1782,1870,web,2200,15.0,eaeefcf45446021f396ff0d63c57983a,2017-03-09,Variant 1,True,0
4,386193,1782,1870,web,2200,15.0,a9790a1f76d7f6f19de78353393023f2,2017-06-13,Variant 2 - video version B,NaN,0


In [6]:
#  reloaded all string data into categories

columns_to_convert = ['source', 'campaign_id', 'name_campaigns', 'opened_email']

for col in columns_to_convert:
    data_raw[col] = pd.to_numeric(data_raw[col], errors='coerce').fillna(0).astype(int)
print(data_raw[columns_to_convert].dtypes)

source            int64
campaign_id       int64
name_campaigns    int64
opened_email      int64
dtype: object


In [7]:
#  Historical behaviour

In [8]:
#  features:

# purchses:
#  if previous campaign led to a purchase
# how many emails ago a user made a purchase
# average num of emails led to purshase
# average interval in days between purshase

# discount:
# was the previous discount bigger
# num of purchase without discount
# discount with the max amount of purchases

# seasonality:
# was the purchase the same day year ago
# weekend

# product:
# have a user bought this item before
# what was the % discount for this item


In [9]:
#  if previous campaign led to a purchase

data_raw['sent_at'] = pd.to_datetime(data_raw['sent_at'])
data_raw = data_raw.sort_values(['user_id', 'sent_at'])
data_raw['last_email'] = data_raw.groupby('user_id')['purchase'].shift(1).fillna(0).astype(int)

In [10]:
# how many emails ago a user made a purchase

data_raw = data_raw.sort_values(['user_id', 'sent_at'])

# for each row, count zeros before first 1 up to that point
data_raw['last_pos_email'] = data_raw.groupby('user_id')['purchase'].transform(
    lambda x: [x[:i+1].argmax() if (x[:i+1] == 1).any() else i+1 
               for i in range(len(x))]
)

In [11]:
# average num of emails led to purshase

# sort values
data_raw = data_raw.sort_values(['user_id', 'sent_at'])

# count cumulative emails and purchases
data_raw['cumcount'] = data_raw.groupby('user_id').cumcount()
data_raw['cum_purchases'] = data_raw.groupby('user_id')['purchase'].cumsum()
data_raw['cum_emails'] = data_raw['cumcount'] - data_raw['cum_purchases']

# average emails per purchase 
data_raw['avg_email_sent'] = data_raw['cum_emails'] / data_raw['cum_purchases']
data_raw['avg_email_sent'] = data_raw['avg_email_sent'].replace([float('inf'), -float('inf')], 0).fillna(0)
data_raw['avg_email_sent'] = data_raw['avg_email_sent'].abs()

# Clean up
data_raw.drop(['cumcount', 'cum_purchases', 'cum_emails'], axis=1, inplace=True)


In [12]:
# average interval in days between purshase

# sort by user_id and sent_at first
data_raw = data_raw.sort_values(['user_id', 'sent_at'])

# get only purchase rows (purchase == 1)
purchase_rows = data_raw[data_raw['purchase'] == 1].copy()

# calculate days between consecutive purchases per user
purchase_rows['prev_purchase_date'] = purchase_rows.groupby('user_id')['sent_at'].shift(1)
purchase_rows['days_since_last_purchase'] = (purchase_rows['sent_at'] - purchase_rows['prev_purchase_date']).dt.days

# calculate cumulative average days between purchases per user
purchase_rows['purchase_count'] = purchase_rows.groupby('user_id').cumcount() + 1
purchase_rows['cumsum_days'] = purchase_rows.groupby('user_id')['days_since_last_purchase'].cumsum()
purchase_rows['avg_days_between_purchases'] = purchase_rows['cumsum_days'] / (purchase_rows['purchase_count'] - 1)

# handle first purchase (no previous purchase to compare)
purchase_rows['avg_days_between_purchases'] = purchase_rows['avg_days_between_purchases'].fillna(0)

# merge to original dataframe
data_raw['avg_days_between_purchases'] = purchase_rows.set_index(purchase_rows.index)['avg_days_between_purchases']

#  fill to propagate the average to all rows for each user
data_raw['avg_days_between_purchases'] = data_raw.groupby('user_id')['avg_days_between_purchases'].fillna(method='ffill').fillna(0)

In [13]:
#  was the previous discount bigger

# sort 
data_raw = data_raw.sort_values(['user_id', 'sent_at'])

# get previous discount% value per user
data_raw['prev_discount_value'] = data_raw.groupby('user_id')['discount%'].shift(1)

# create prev_discount column: 1 if previous discount is bigger, 0 otherwise
data_raw['prev_discount'] = (data_raw['prev_discount_value'] > data_raw['discount%']).astype(int)

# handle first row per user (no previous value) - set to 0
data_raw['prev_discount'] = data_raw['prev_discount'].fillna(0).astype(int)

# clean up helper 
data_raw.drop('prev_discount_value', axis=1, inplace=True)

In [14]:
# num of purchase without discount

# Sort
data_raw = data_raw.sort_values(['user_id', 'sent_at'])

# Create a flag for purchases without discount (discount% == 0 and purchase == 1)
data_raw['purchase_no_discount'] = ((data_raw['discount%'] == 0) & (data_raw['purchase'] == 1)).astype(int)

# Calculate cumulative count of purchases without discount per user
data_raw['no_discount'] = data_raw.groupby('user_id')['purchase_no_discount'].cumsum()

# Clean up helper column
data_raw.drop('purchase_no_discount', axis=1, inplace=True)

In [15]:
# discount with the max amount of purchases



data_raw = data_raw.sort_values(['user_id', 'sent_at'])

#  Find discount with max purchases per user 
data_raw['max_purchase_discount'] = data_raw.groupby('user_id').apply(
    lambda group: group.apply(
        lambda row: group.loc[:row.name][group.loc[:row.name, 'purchase'] == 1]['discount%'].value_counts().idxmax() 
        if len(group.loc[:row.name][group.loc[:row.name, 'purchase'] == 1]) > 0 else 0, 
        axis=1
    )
).values

In [16]:
# was the purchase the same day year ago

# Extract day and month
data_raw['day_month'] = data_raw['sent_at'].dt.strftime('%m-%d')

# Initialize same_day column
data_raw['same_day'] = 0

# Only check for purchase rows
purchase_mask = data_raw['purchase'] == 1
purchase_data = data_raw[purchase_mask].copy()

# Group by user and day_month, then check if there are multiple purchases
grouped = purchase_data.groupby(['user_id', 'day_month']).apply(
    lambda x: x.index[1:] if len(x) > 1 else []
).explode()

# Set same_day = 1 for indices where there were previous purchases on same day-month
data_raw.loc[grouped.dropna(), 'same_day'] = 1

# Clean up helper column
data_raw.drop('day_month', axis=1, inplace=True)

In [17]:
# weekend


# Create weekend column: 1 if purchase made on weekend, 0 otherwise
data_raw['weekend'] = (
    (data_raw['purchase'] == 1) & 
    (data_raw['sent_at'].dt.weekday >= 5)  # Saturday=5, Sunday=6
).astype(int)

In [18]:
#avg a user bought this item before


# Count how many times user bought this product before (cumulative count - 1)
data_raw['bought_before'] = data_raw.groupby(['user_id', 'product_id']).cumcount()


In [19]:
# what was the % discount for this item



# Calculate cumulative average discount per product_id per row
data_raw['avg_item_discount'] = data_raw.groupby('product_id')['discount%'].transform(lambda x: x.expanding().mean())

In [20]:
data_raw.to_csv('../Project_datasets/feature_dataset.csv')

In [22]:
data_raw.columns

Index(['user_id', 'product_id', 'total_cost_pennies', 'source',
       'total_cost_pre_discount_pennies', 'discount%', 'campaign_id',
       'sent_at', 'name_campaigns', 'opened_email', 'purchase', 'last_email',
       'last_pos_email', 'avg_email_sent', 'avg_days_between_purchases',
       'prev_discount', 'no_discount', 'max_purchase_discount', 'same_day',
       'weekend', 'bought_before', 'avg_item_discount'],
      dtype='object')